In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
from src.tables import get_machines
from src.utils import df_na_vals

In [3]:
raw = get_machines()

In [4]:
cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = raw.cpu_name.str.replace('Core™', '')
raw.cpu_name = cpu_name
raw['cores'] = (raw.cpu_cores / raw.num_gpus).fillna(0).astype(int)
raw['ram'] = (raw.cpu_ram / raw.num_gpus / 1024).round().astype(int)
raw['dph'] = raw.dph_base / raw.num_gpus
raw['disk'] = (raw.disk_space / raw.num_gpus).astype(int)
raw['rented_fraq'] = '[' + raw.num_gpus_rented.astype(str) + "/" + raw.num_gpus.astype(str) + ']'
raw['country'] = raw.location.dropna().apply(lambda loc: loc['country'])
# raw['rented'] = raw.num_gpus_rented > 0
raw['rented'] = raw.num_gpus_rented == raw.num_gpus
raw['min_fraq'] = raw.min_chunk/raw.num_gpus

raw.dlperf = raw.dlperf.round(1)
raw.dlperf_per_dphtotal = raw.dlperf_per_dphtotal.round(1)
raw.disk_bw = raw.disk_bw.round()
raw.duration = pd.to_timedelta(raw.duration * 1e9).dt.days
raw.inet_down = raw.inet_down.fillna(0).astype(int)
raw.inet_up = raw.inet_up.fillna(0).astype(int)
raw.inet_up_cost = (raw.inet_up_cost * 1000).fillna(0).astype(int)
raw.inet_down_cost = (raw.inet_down_cost * 1000).fillna(0).astype(int)


# raw.end_date = (pd.to_datetime(raw.end_date * 1e9) - pd.Timestamp.now()).dt.days
raw.end_date = pd.to_datetime(raw.end_date * 1e9)
raw.start_date = pd.to_datetime(raw.start_date * 1e9)
# raw.set_index('machine_id', inplace=True)

C:\Users\ergot\AppData\Local\Temp\ipykernel_2044\182450783.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = raw.cpu_name.str.replace(r'^\d\dth Gen ','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_2044\182450783.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'^AMD','')
C:\Users\ergot\AppData\Local\Temp\ipykernel_2044\182450783.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  cpu_name = cpu_name.str.replace(r'Processor$','')


In [5]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'dlperf_per_dphtotal', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability2', 'duration', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'location']

raw.query('dph >= 0.5')[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
195,H100 PCIE,[4/4],3.5,0.20,27.7,64,244,3301,7769.0,4,...,2,12.2,535.104.05,4,16,NaN,15.4,EPYC 7763 64C,ROME2D32GM,"{'country': 'TW', 'location': 'New Taipei', 'l..."
494,H100 PCIE,[2/2],3.0,0.40,34.0,64,118,1275,4296.0,1,...,0,12.2,535.54.03,4,16,NaN,21.7,EPYC 7742 64C,MZ52,"{'country': 'US', 'location': 'Ogden, Utah', '..."
1310,RTX 4090,[0/1],2.5,0.40,15.3,128,504,3375,5657.0,1,...,2,12.0,525.125.06,4,16,NaN,25.2,EPYC 7B13 64C,MZ32,"{'country': 'JP', 'location': 'Tokyo', 'lat': ..."
211,RTX 3090,[0/4],2.0,0.20,11.7,16,31,411,3475.0,1,...,20,12.2,535.54.03,3,16,NaN,6.1,Ryzen TR PRO 3975WX 32Cs,Pro WS WRX80E,"{'country': 'NO', 'location': 'Drammen, Viken'..."
608,A100 SXM4,[6/8],1.8,0.15,32.9,32,250,1454,6848.0,1,...,2,12.2,535.104.05,4,16,NaN,25.1,EPYC 7763 64C,H12DGO,"{'country': 'SI', 'location': 'Ljubljana', 'la..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,A40,[4/5],0.5,0.40,61.7,3,50,39,1655.0,1,...,10,12.0,525.125.06,3,8,1.0,11.9,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
875,A40,[4/5],0.5,0.40,61.7,3,50,38,2028.0,1,...,10,12.0,525.125.06,3,16,1.0,11.9,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
874,A40,[4/5],0.5,0.40,62.0,3,50,28,1668.0,1,...,0,12.0,525.125.06,3,8,1.0,11.9,EPYC 7252 8C,Default string,"{'country': 'SE', 'location': 'Umeå, Västerbot..."
869,RTX 4090,[0/2],0.5,0.40,142.5,6,31,665,2819.0,1,...,2,12.2,535.86.05,4,8,NaN,12.7,Ryzen 5 7600X 6C,ProArt B650,"{'country': 'CA', 'location': 'North Vancouver..."


In [6]:
raw.query('num_gpus == 14')[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
400,RTX 4090,[9/14],0.45,0.1,157.9,13,72,428,5272.0,1,...,5,12.2,535.86.05,4,8,NaN,12.6,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Pingzhen Distri..."
412,RTX 4090,[10/14],0.40,0.1,173.7,13,36,450,3535.0,1,...,5,12.2,535.86.05,4,8,NaN,12.3,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Pingzhen Distri..."
432,RTX 4090,[0/14],0.50,0.1,57.2,13,72,457,4917.0,14,...,0,12.2,535.86.05,4,8,NaN,12.6,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Pingzhen Distri..."
439,RTX 4090,[14/14],0.40,0.1,64.9,13,72,452,5312.0,14,...,5,12.2,535.86.05,4,8,NaN,12.8,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Pingzhen Distri..."
619,RTX 4090,[14/14],0.55,0.4,100.5,18,36,202,1571.0,1,...,0,12.0,525.125.06,4,8,NaN,12.6,EPYC 7H12 64C,ROME2D32GM,"{'country': 'US', 'location': 'Oklahoma City, ..."
621,RTX 4090,[8/14],0.60,0.4,120.9,18,36,71,3412.0,1,...,2,12.2,535.54.03,4,4,NaN,12.5,EPYC 7662 64C,ROME2D32GM,"{'country': 'US', 'location': 'Oklahoma City, ..."
856,RTX 4090,[14/14],0.50,0.2,60.1,13,36,375,5463.0,14,...,0,12.0,525.105.17,4,8,NaN,12.5,EPYC 7R12 48C,ROME2D32GM,"{'country': 'TW', 'location': 'Zhongli Distric..."
895,RTX 4090,[0/14],0.55,0.2,59.5,18,36,87,4711.0,14,...,2,12.0,525.105.17,4,8,NaN,12.8,EPYC 7742 64C,ROME2D32GM,"{'country': 'TW', 'location': 'Taoyuan Distric..."


In [7]:
raw[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,dlperf_per_dphtotal,cores,ram,disk,disk_bw,min_chunk,...,inet_down_cost,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,location
9,RTX 4090,[2/2],0.150,0.05,468.3,16,63,1629,5904.0,1,...,0,12.2,535.104.05,3,8,NaN,6.4,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
89,RTX 3080 Ti,[0/4],0.100,0.05,250.0,18,63,812,3088.0,1,...,0,12.0,525.125.06,3,8,NaN,5.5,Xeon® E5-2696 v3,X99,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
119,RTX 4090,[0/2],0.400,0.40,189.5,64,31,825,4845.0,1,...,0,12.2,535.104.05,4,16,NaN,24.8,EPYC 7B13 64C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
122,RTX 3090,[1/1],0.270,0.30,125.3,128,252,808,480.0,1,...,0,12.2,535.54.03,4,16,NaN,22.3,EPYC 7742 64C,ROMED8,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
140,RTX 3080,[0/2],0.400,0.40,48.7,2,16,795,3407.0,2,...,2,12.0,525.125.06,3,16,NaN,12.5,Core™ i5-7400,B250 Gaming K4,"{'country': 'RU', 'location': 'Vologda, Vologd..."
229,RTX 4090,[2/2],0.150,0.40,473.1,18,63,1607,3203.0,1,...,0,12.0,525.125.06,3,16,NaN,11.7,Xeon® E5-2696 v3,X99,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
256,RTX 3090,[0/1],0.100,0.40,270.8,16,31,34,2514.0,1,...,0,12.2,535.86.05,3,16,NaN,11.3,Xeon® E5-2689 0,E5,"{'country': 'RU', 'location': 'Yaroslavl, Yaro..."
289,RTX 3080,[5/6],0.070,0.40,64.0,2,3,62,425.0,1,...,1,12.2,535.86.05,1,1,NaN,0.1,Core™ i5-10400F,H510 Pro BTC,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
434,RTX 3090,[1/1],0.250,0.40,131.4,12,31,744,721.0,1,...,3,12.2,535.54.03,4,16,NaN,24.7,Ryzen 5 3600 6C,B550M DS3H,"{'country': 'RU', 'location': 'Moscow', 'lat':..."
502,RTX 3090,[3/3],0.198,0.05,156.6,10,21,53,2883.0,1,...,0,12.0,525.125.06,4,8,NaN,12.8,Ryzen 9 5950X 16C,ROG STRIX X570,"{'country': 'RU', 'location': 'Maslyaninskiy, ..."
